In [4]:
import google.auth
from google.cloud import aiplatform, storage
from google.auth.credentials import Credentials

import pandas as pd
import os

In [5]:
credentials, project = google.auth.default()

In [6]:
PATH = os.path.realpath('../data/train_data_s3')

In [7]:
pathes = list()
for classname in os.listdir(PATH):
    for filename in os.listdir(os.path.join(PATH, classname)):
        pathes.append(f'{classname}/{filename}')

In [8]:
df = pd.DataFrame({'path': pathes})

In [9]:
df['label'] = df['path'].str.split('/').str[0]
df['path'] = df['path'].apply(lambda x: f'gs://bird-project-mlops-vertex-data/train_data_s3/{x}')

In [10]:
df.to_csv('../data/dataset.csv', index=False)

In [11]:
storage_client = storage.Client.from_service_account_json('../data/bird-project-mlops-vertex-ab4a1e84f536.json')

In [12]:
def download_blob(bucket_name, source_blob_name, destination_file_name, storage_client):
    """Downloads a blob from the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your GCS object
    # source_blob_name = "storage-object-name"

    # The path to which the file should be downloaded
    # destination_file_name = "local/path/to/file"

    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)


In [13]:
from google.cloud.storage import Client, transfer_manager

In [14]:
def download_bucket_with_transfer_manager(
        bucket_name, storage_client, destination_directory="", workers=8, max_results=10000, 
    ):
        """Download all of the blobs in a bucket, concurrently in a process pool.

        The filename of each blob once downloaded is derived from the blob name and
        the `destination_directory `parameter. For complete control of the filename
        of each blob, use transfer_manager.download_many() instead.

        Directories will be created automatically as needed, for instance to
        accommodate blob names that include slashes.
        """

        # The ID of your GCS bucket
        # bucket_name = "your-bucket-name"

        # The directory on your computer to which to download all of the files. This
        # string is prepended (with os.path.join()) to the name of each blob to form
        # the full path. Relative paths and absolute paths are both accepted. An
        # empty string means "the current working directory". Note that this
        # parameter allows accepts directory traversal ("../" etc.) and is not
        # intended for unsanitized end user input.
        # destination_directory = ""

        # The maximum number of processes to use for the operation. The performance
        # impact of this value depends on the use case, but smaller files usually
        # benefit from a higher number of processes. Each additional process occupies
        # some CPU and memory resources until finished. Threads can be used instead
        # of processes by passing `worker_type=transfer_manager.THREAD`.
        # workers=8

        # The maximum number of results to fetch from bucket.list_blobs(). This
        # sample code fetches all of the blobs up to max_results and queues them all
        # for download at once. Though they will still be executed in batches up to
        # the processes limit, queueing them all at once can be taxing on system
        # memory if buckets are very large. Adjust max_results as needed for your
        # system environment, or set it to None if you are sure the bucket is not
        # too large to hold in memory easily.
        # max_results=1000
        bucket = storage_client.bucket(bucket_name)

        print("Bucket created")

        blob_names = [blob.name for blob in bucket.list_blobs(max_results=max_results)]

        print("Blobs listed")

        transfer_manager.download_many_to_path(
            bucket, blob_names, destination_directory=destination_directory, max_workers=workers
        )

        print("Blobs downloaded")

In [16]:
download_bucket_with_transfer_manager('bird-project-mlops-vertex-data', storage_client, max_results=30, destination_directory='../data/train_data_gs_sample')

Bucket created
Blobs listed
Blobs downloaded


In [19]:
download_blob('bird-project-mlops-vertex-data', 'dataset.csv', '../data/dataset-copy.csv', storage_client)

In [9]:
aiplatform.init(
    project='bird-project-mlops-vertex',
    location='us-central1',
    staging_bucket='gs://bird-project-mlops-vertex-bucket',
    credentials=credentials,
)

In [34]:
dataset_id = 'birds-dataset-2'

In [ ]:
dataset = aiplatform.ImageDataset(dataset_id)

In [14]:
model_registry = aiplatform.models.ModelRegistry(model="projects/bird-project-mlops-vertex/locations/us-central1/models/1")
model_registry.list_versions()


Getting versions for projects/bird-project-mlops-vertex/locations/us-central1/models/1


ServiceUnavailable: 503 Getting metadata from plugin failed with error: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})